In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# country_df = pd.read_csv("data\Country.csv")
# country_history_df = pd.read_csv("data\Country_History.csv")
county_history_df = pd.read_csv("data\County_History.csv")
# metro_df = pd.read_csv("data\Metro.csv")
# metro_history_df = pd.read_csv("data\Metro_History.csv")
state_history_df = pd.read_csv("data\State_History.csv")
# zip_df = pd.read_csv("data\Zip.csv")
# zip_history_df = pd.read_csv("data\Zip_History.csv")

In [3]:
county_history_df

,month_date_yyyymm,county_fips,county_name,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,202506,45079,"richland, sc",289900.0,0.0000,-0.0333,1241.0,0.0891,0.2882,37.0,...,359136.0,-0.0120,-0.0324,2289.0,0.0228,0.1708,0.8460,-0.1189,-0.2090,0.0
1,202506,40085,"love, ok",323700.0,0.0195,0.2948,50.0,0.3026,0.4559,66.0,...,388866.0,-0.0454,0.0871,56.0,0.2065,0.3875,0.1414,-0.0691,-0.0351,0.0
2,202506,39057,"greene, oh",364000.0,0.0400,0.0196,217.0,-0.0069,-0.0378,36.0,...,402502.0,0.0319,0.0041,539.0,-0.0182,-0.0037,1.4896,-0.0379,0.1118,0.0
3,202506,19073,"greene, ia",176250.0,-0.0042,0.0602,34.0,-0.1410,0.0806,68.0,...,222163.0,-0.0556,-0.0108,46.0,-0.1415,0.1375,0.3582,-0.0008,0.0679,0.0
4,202506,31181,"webster, ne",174750.0,-0.1152,0.2527,13.0,-0.0714,0.4444,42.0,...,353275.0,-0.0143,0.7768,21.0,0.0789,0.4643,0.5769,0.2198,0.0214,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335188,201607,48233,"hutchinson, tx",105900.0,NaN,NaN,79.0,NaN,NaN,61.0,...,139006.0,NaN,NaN,121.0,NaN,NaN,0.5316,NaN,NaN,NaN
335189,201607,37035,"catawba, nc",209900.0,NaN,NaN,887.0,NaN,NaN,73.0,...,294663.0,NaN,NaN,948.0,NaN,NaN,0.0981,NaN,NaN,NaN
335190,201607,4015,"mohave, az",234900.0,NaN,NaN,1606.0,NaN,NaN,83.0,...,292296.0,NaN,NaN,1737.0,NaN,NaN,0.0834,NaN,NaN,NaN
335191,201607,37123,"montgomery, nc",329000.0,NaN,NaN,255.0,NaN,NaN,109.0,...,405866.0,NaN,NaN,279.0,NaN,NaN,0.0902,NaN,NaN,NaN


In [4]:
state_history_df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,202507,Montana,MT,649900,0.0000,-0.0175,6116,0.0597,0.1494,59,...,1154428,0.0346,-0.0165,7749,0.0318,0.1257,0.2699,-0.0315,-0.0263,0.0
1,202507,Arkansas,AR,305000,0.0000,0.0224,12652,0.0239,0.2615,61,...,406603,-0.0062,0.0244,17218,0.0058,0.1854,0.3731,-0.0243,-0.0883,0.0
2,202507,Connecticut,CT,549900,-0.0211,0.0337,5120,0.0509,0.1429,39,...,1084062,-0.0242,-0.0002,10652,-0.0171,0.0326,1.0828,-0.1348,-0.2268,0.0
3,202507,Massachusetts,MA,797000,-0.0031,-0.0031,11283,-0.0341,0.2627,43,...,1418895,-0.0020,-0.0381,13412,-0.0494,0.2243,0.1961,-0.0131,-0.0418,0.0
4,202507,Alaska,AK,445450,-0.0079,0.0361,1970,0.0890,0.0407,51,...,540136,-0.0050,0.0699,3062,0.0533,0.0076,0.5566,-0.0523,-0.0469,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5554,201607,Mississippi,MS,171500,NaN,NaN,13196,NaN,NaN,80,...,227962,NaN,NaN,14797,NaN,NaN,0.1228,NaN,NaN,NaN
5555,201607,Rhode Island,RI,329900,NaN,NaN,4444,NaN,NaN,53,...,542692,NaN,NaN,5523,NaN,NaN,0.2480,NaN,NaN,NaN
5556,201607,Wisconsin,WI,188800,NaN,NaN,37250,NaN,NaN,64,...,260082,NaN,NaN,41623,NaN,NaN,0.1266,NaN,NaN,NaN
5557,201607,Colorado,CO,419000,NaN,NaN,24590,NaN,NaN,45,...,789807,NaN,NaN,30981,NaN,NaN,0.2612,NaN,NaN,NaN


In [5]:
state_history_df['month_date_yyyymm'] = pd.to_datetime(state_history_df['month_date_yyyymm'].astype(str), format='%Y%m')

In [6]:
state_history_df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,...,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
0,2025-07-01,Montana,MT,649900,0.0000,-0.0175,6116,0.0597,0.1494,59,...,1154428,0.0346,-0.0165,7749,0.0318,0.1257,0.2699,-0.0315,-0.0263,0.0
1,2025-07-01,Arkansas,AR,305000,0.0000,0.0224,12652,0.0239,0.2615,61,...,406603,-0.0062,0.0244,17218,0.0058,0.1854,0.3731,-0.0243,-0.0883,0.0
2,2025-07-01,Connecticut,CT,549900,-0.0211,0.0337,5120,0.0509,0.1429,39,...,1084062,-0.0242,-0.0002,10652,-0.0171,0.0326,1.0828,-0.1348,-0.2268,0.0
3,2025-07-01,Massachusetts,MA,797000,-0.0031,-0.0031,11283,-0.0341,0.2627,43,...,1418895,-0.0020,-0.0381,13412,-0.0494,0.2243,0.1961,-0.0131,-0.0418,0.0
4,2025-07-01,Alaska,AK,445450,-0.0079,0.0361,1970,0.0890,0.0407,51,...,540136,-0.0050,0.0699,3062,0.0533,0.0076,0.5566,-0.0523,-0.0469,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5554,2016-07-01,Mississippi,MS,171500,NaN,NaN,13196,NaN,NaN,80,...,227962,NaN,NaN,14797,NaN,NaN,0.1228,NaN,NaN,NaN
5555,2016-07-01,Rhode Island,RI,329900,NaN,NaN,4444,NaN,NaN,53,...,542692,NaN,NaN,5523,NaN,NaN,0.2480,NaN,NaN,NaN
5556,2016-07-01,Wisconsin,WI,188800,NaN,NaN,37250,NaN,NaN,64,...,260082,NaN,NaN,41623,NaN,NaN,0.1266,NaN,NaN,NaN
5557,2016-07-01,Colorado,CO,419000,NaN,NaN,24590,NaN,NaN,45,...,789807,NaN,NaN,30981,NaN,NaN,0.2612,NaN,NaN,NaN


In [7]:
def customDescription(df: pd.DataFrame, numeric_only: bool = False):
    if numeric_only:
        df = df.select_dtypes(include=np.number)
    
    desc = pd.DataFrame(index=df.columns.to_list())
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['null'] = df.isnull().sum()
    
    # Calculate mode and handle multiple modes
    modes = df.mode()
    desc['mode'] = np.nan  # Default to NaN
    for col in df.columns:
        if len(modes[col].dropna()) == 1:  # Single mode exists
            desc.loc[col, 'mode'] = modes[col].iloc[0]
        else:  # Multiple modes
            desc.loc[col, 'mode'] = np.nan

    # Calculate least frequent value
    desc['least_frequent'] = np.nan  # Default to NaN
    for col in df.columns:
        value_counts = df[col].value_counts(dropna=False)
        if not value_counts.empty:
            least_freq_count = value_counts.min()  # Find the minimum frequency
            least_freq_values = value_counts[value_counts == least_freq_count].index
            
            if len(least_freq_values) == 1:  # If exactly one least frequent value exists
                desc.loc[col, 'least_frequent'] = least_freq_values[0]
            else:  # Multiple least frequent values
                desc.loc[col, 'least_frequent'] = np.nan
    
    # Handle numeric columns
    numeric_cols = df.select_dtypes(include=np.number)
    if not numeric_cols.empty:
        numeric_desc = numeric_cols.describe().T.drop(columns=['count', 'std', '25%', '50%', '75%'], axis=1)
        for col in numeric_cols.columns:
            desc.loc[col, 'mean'] = numeric_desc.loc[col, 'mean']
            desc.loc[col, 'min'] = numeric_desc.loc[col, 'min']
            desc.loc[col, 'max'] = numeric_desc.loc[col, 'max']
    
    # Handle datetime columns
    datetime_cols = df.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]'])
    for col in datetime_cols.columns:
        desc.loc[col, 'min'] = df[col].min()
        desc.loc[col, 'max'] = df[col].max()
    
    return desc

In [8]:
df = state_history_df.copy()

In [9]:
customDescription(df)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,datetime64[ns],5559,109,0,NaN,NaN,NaN,2016-07-01 00:00:00,2025-07-01 00:00:00
state,object,5559,51,0,NaN,NaN,NaN,NaN,NaN
state_id,object,5559,51,0,NaN,NaN,NaN,NaN,NaN
median_listing_price,int64,5559,2633,0,299900.0000,NaN,361098.491995,129913.0,886500.0
median_listing_price_mm,float64,4947,1001,612,0.0000,NaN,0.005178,-0.1038,0.1135
median_listing_price_yy,float64,4947,2159,612,0.0000,NaN,0.068167,-0.2071,0.4101
active_listing_count,int64,5559,5076,0,NaN,NaN,17686.877856,720.0,182589.0
active_listing_count_mm,float64,4947,2477,612,NaN,NaN,0.000574,-0.3074,0.6116
active_listing_count_yy,float64,4947,3858,612,NaN,NaN,-0.009808,-0.7173,2.3532
median_days_on_market,int64,5559,130,0,51.0000,NaN,58.509624,13.0,157.0


In [10]:
pd.set_option('display.max_columns', None)

In [11]:
df[df['state_id'] == 'WV']

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy,quality_flag
37,2025-07-01,West Virginia,WV,270000,-0.0182,0.1192,3808,0.0366,0.1582,53,0.1053,-0.0187,1900,-0.0031,0.1337,42,-0.1923,-0.4167,0.0049,-0.0016,-0.0044,1296,0.0554,0.1634,0.1535,0.0005,0.0073,2448.0,-0.0430,-0.0724,147,-0.0098,0.0778,1792,-0.0058,0.0234,340222,-0.0132,0.0399,6248,0.0039,0.0563,0.6429,-0.0535,-0.1598,0.0
89,2025-06-01,West Virginia,WV,275000,0.0189,0.1134,3674,0.0868,0.1556,48,0.0556,-0.0306,1906,0.0203,0.0732,52,-0.1875,-0.4348,0.0065,-0.0017,-0.0053,1228,0.1164,0.2429,0.1529,0.0138,0.0229,2558.0,-0.0104,-0.0237,148,0.0104,0.0888,1803,0.0059,0.0207,344764,0.0138,0.0448,6224,0.0410,0.0700,0.6963,-0.0685,-0.1278,0.0
120,2025-05-01,West Virginia,WV,269900,0.0228,0.1251,3380,0.1030,0.1558,45,-0.0323,-0.0426,1868,0.0021,0.0950,64,0.2308,0.0667,0.0082,0.0011,0.0001,1100,0.2443,0.3189,0.1392,0.0203,0.0259,2585.0,0.0459,-0.0458,147,0.0133,0.0985,1792,0.0150,0.0175,340074,0.0248,0.0342,5979,0.0819,0.0572,0.7648,-0.0417,-0.1615,0.0
195,2025-04-01,West Virginia,WV,263881,0.0188,0.0995,3065,0.0495,0.0808,47,-0.2846,-0.0792,1864,0.1768,0.0664,52,-0.4583,-0.5873,0.0070,-0.0075,-0.0112,884,0.1818,0.0351,0.1188,0.0017,-0.0008,2472.0,0.1780,-0.0104,145,0.0082,0.0870,1766,0.0066,0.0193,331834,0.0272,0.0107,5527,0.1031,0.0392,0.8065,0.0880,-0.0743,0.0
214,2025-03-01,West Virginia,WV,259000,0.0486,0.0837,2920,-0.0363,0.0607,65,-0.1772,0.1017,1584,0.3846,-0.0594,96,0.1429,0.0000,0.0146,0.0002,0.0003,748,0.1000,0.0054,0.1171,0.0068,0.0023,2098.0,0.1180,-0.0446,144,0.0271,0.1150,1754,0.0218,-0.0048,323062,0.0377,-0.0055,5010,0.0173,0.0089,0.7185,0.0992,-0.0792,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5321,2016-11-01,West Virginia,WV,155925,NaN,NaN,9084,NaN,NaN,95,NaN,NaN,1556,NaN,NaN,46,NaN,NaN,0.0039,NaN,NaN,1266,NaN,NaN,0.1082,NaN,NaN,1081.0,NaN,NaN,89,NaN,NaN,1760,NaN,NaN,212205,NaN,NaN,10144,NaN,NaN,0.1190,NaN,NaN,NaN
5370,2016-10-01,West Virginia,WV,159000,NaN,NaN,9442,NaN,NaN,90,NaN,NaN,1848,NaN,NaN,64,NaN,NaN,0.0054,NaN,NaN,1532,NaN,NaN,0.1235,NaN,NaN,1212.0,NaN,NaN,90,NaN,NaN,1764,NaN,NaN,212698,NaN,NaN,10635,NaN,NaN,0.1284,NaN,NaN,NaN
5415,2016-09-01,West Virginia,WV,159875,NaN,NaN,9606,NaN,NaN,87,NaN,NaN,1734,NaN,NaN,64,NaN,NaN,0.0050,NaN,NaN,1666,NaN,NaN,0.1317,NaN,NaN,1225.0,NaN,NaN,90,NaN,NaN,1773,NaN,NaN,213569,NaN,NaN,10807,NaN,NaN,0.1275,NaN,NaN,NaN
5502,2016-08-01,West Virginia,WV,161350,NaN,NaN,9591,NaN,NaN,82,NaN,NaN,2046,NaN,NaN,70,NaN,NaN,0.0054,NaN,NaN,1780,NaN,NaN,0.1378,NaN,NaN,1291.0,NaN,NaN,90,NaN,NaN,1781,NaN,NaN,215415,NaN,NaN,10867,NaN,NaN,0.1346,NaN,NaN,NaN


In [12]:
# Sort by state and month
df = df.sort_values(["state", "month_date_yyyymm"])

def recompute_mm(df, base_col, mm_col):
    # Compute pct_change
    df[mm_col] = df.groupby("state")[base_col].pct_change()

    # Replace inf values with the base_col (new month value)
    inf_mask = df[mm_col].replace([np.inf, -np.inf], np.nan).isna() & df[base_col].notna()
    df.loc[inf_mask, mm_col] = df.loc[inf_mask, base_col]

    return df

In [13]:
# You can loop over all *_mm cols systematically:
base_to_mm = {
    "median_listing_price": "median_listing_price_mm",
    "active_listing_count": "active_listing_count_mm",
    "median_days_on_market": "median_days_on_market_mm",
    "new_listing_count": "new_listing_count_mm",
    "price_increased_count": "price_increased_count_mm",
    "price_increased_share": "price_increased_share_mm",
    "price_reduced_count": "price_reduced_count_mm",
    "price_reduced_share": "price_reduced_share_mm",
    "pending_listing_count": "pending_listing_count_mm",
    "median_listing_price_per_square_foot": "median_listing_price_per_square_foot_mm",
    "median_square_feet": "median_square_feet_mm",
    "average_listing_price": "average_listing_price_mm",
    "total_listing_count": "total_listing_count_mm",
    "pending_ratio": "pending_ratio_mm"
}

for base, mm in base_to_mm.items():
    if base in df.columns and mm in df.columns:
        df = recompute_mm(df, base, mm)

In [14]:
df.reset_index(drop=True, inplace=True)
df.drop(columns=['quality_flag'], inplace=True, errors='ignore')
df

,month_date_yyyymm,state,state_id,median_listing_price,median_listing_price_mm,median_listing_price_yy,active_listing_count,active_listing_count_mm,active_listing_count_yy,median_days_on_market,median_days_on_market_mm,median_days_on_market_yy,new_listing_count,new_listing_count_mm,new_listing_count_yy,price_increased_count,price_increased_count_mm,price_increased_count_yy,price_increased_share,price_increased_share_mm,price_increased_share_yy,price_reduced_count,price_reduced_count_mm,price_reduced_count_yy,price_reduced_share,price_reduced_share_mm,price_reduced_share_yy,pending_listing_count,pending_listing_count_mm,pending_listing_count_yy,median_listing_price_per_square_foot,median_listing_price_per_square_foot_mm,median_listing_price_per_square_foot_yy,median_square_feet,median_square_feet_mm,median_square_feet_yy,average_listing_price,average_listing_price_mm,average_listing_price_yy,total_listing_count,total_listing_count_mm,total_listing_count_yy,pending_ratio,pending_ratio_mm,pending_ratio_yy
0,2016-07-01,Alabama,AL,185900,185900.000000,NaN,30468,30468.000000,NaN,78,78.000000,NaN,7756,7756.000000,NaN,368,368.000000,NaN,0.0094,0.009400,NaN,6132,6132.000000,NaN,0.1553,0.155300,NaN,1857.0,1857.000000,NaN,92,92.000000,NaN,1976,1976.000000,NaN,250718,250718.000000,NaN,32344,32344.000000,NaN,0.0609,0.060900,NaN
1,2016-08-01,Alabama,AL,185000,-0.004841,NaN,29926,-0.017789,NaN,83,0.064103,NaN,6884,-0.112429,NaN,452,0.228261,NaN,0.0117,0.244681,NaN,5864,-0.043705,NaN,0.1526,-0.017386,NaN,1905.0,0.025848,NaN,92,0.000000,NaN,1974,-0.001012,NaN,250649,-0.000275,NaN,31818,-0.016263,NaN,0.0636,0.044335,NaN
2,2016-09-01,Alabama,AL,184900,-0.000541,NaN,29557,-0.012330,NaN,85,0.024096,NaN,6676,-0.030215,NaN,424,-0.061947,NaN,0.0112,-0.042735,NaN,5790,-0.012619,NaN,0.1529,0.001966,NaN,1736.0,-0.088714,NaN,92,0.000000,NaN,1959,-0.007599,NaN,248452,-0.008765,NaN,31277,-0.017003,NaN,0.0587,-0.077044,NaN
3,2016-10-01,Alabama,AL,182795,-0.011385,NaN,28923,-0.021450,NaN,88,0.035294,NaN,6096,-0.086878,NaN,520,0.226415,NaN,0.0142,0.267857,NaN,5268,-0.090155,NaN,0.1447,-0.053630,NaN,1617.0,-0.068548,NaN,91,-0.010870,NaN,1950,-0.004594,NaN,250271,0.007321,NaN,30527,-0.023979,NaN,0.0559,-0.047700,NaN
4,2016-11-01,Alabama,AL,180500,-0.012555,NaN,27925,-0.034505,NaN,93,0.056818,NaN,5332,-0.125328,NaN,384,-0.261538,NaN,0.0102,-0.281690,NaN,3946,-0.250949,NaN,0.1061,-0.266759,NaN,1452.0,-0.102041,NaN,91,0.000000,NaN,1949,-0.000513,NaN,247238,-0.012119,NaN,29367,-0.037999,NaN,0.0520,-0.069767,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5554,2025-03-01,Wyoming,WY,459900,0.000381,0.0243,1826,-0.004362,0.1127,68,-0.227273,-0.0556,784,0.319865,0.1951,12,12.000000,-0.2500,0.0041,0.004100,-0.0018,332,0.092105,0.4310,0.1058,-0.034672,0.0271,502.0,0.154023,-0.0652,208,-0.004785,0.0800,2176,0.002303,-0.0357,1166754,-0.026201,0.2582,2318,0.029307,0.0657,0.2749,0.159916,-0.0523
5555,2025-04-01,Wyoming,WY,476000,0.035008,0.0462,1945,0.065170,0.1421,57,-0.161765,-0.1172,942,0.201531,0.1135,16,0.333333,-0.2000,0.0046,0.121951,-0.0019,388,0.168675,0.2357,0.1135,0.072779,0.0121,597.0,0.189243,0.0093,209,0.004808,0.0716,2204,0.012868,-0.0267,1108065,-0.050301,0.2097,2535,0.093615,0.1128,0.3067,0.115678,-0.0404
5556,2025-05-01,Wyoming,WY,495000,0.039916,0.0623,2208,0.135219,0.1820,51,-0.105263,0.0000,1028,0.091295,0.1762,12,-0.250000,0.0000,0.0033,-0.282609,-0.0004,520,0.340206,0.3000,0.1354,0.192952,0.0166,615.0,0.030151,-0.0144,211,0.009569,0.0443,2216,0.005445,-0.0353,1064091,-0.039685,0.1772,2816,0.110848,0.1373,0.2785,-0.091947,-0.0555
5557,2025-06-01,Wyoming,WY,496741,0.003517,0.0458,2519,0.140851,0.1644,50,-0.019608,0.0638,1146,0.114786,0.1506,14,0.166667,0.7500,0.0032,-0.030303,0.0011,682,0.311538,0.2724,0.1597,0.179468,0.0161,649.0,0.055285,0.0973,217,0.028436,0.0210,2209,-0.003159,-0.0222,1166952,0.096666,0.1760,3156

In [15]:
customDescription(df)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,datetime64[ns],5559,109,0,NaN,NaN,NaN,2016-07-01 00:00:00,2025-07-01 00:00:00
state,object,5559,51,0,NaN,NaN,NaN,NaN,NaN
state_id,object,5559,51,0,NaN,NaN,NaN,NaN,NaN
median_listing_price,int64,5559,2633,0,299900.0000,NaN,361098.491995,129913.0,886500.0
median_listing_price_mm,float64,5559,4830,0,0.0000,NaN,2454.236641,-0.103768,649000.0
median_listing_price_yy,float64,4947,2159,612,0.0000,NaN,0.068167,-0.2071,0.4101
active_listing_count,int64,5559,5076,0,NaN,NaN,17686.877856,720.0,182589.0
active_listing_count_mm,float64,5559,5553,0,NaN,NaN,263.354728,-0.307421,136780.0
active_listing_count_yy,float64,4947,3858,612,NaN,NaN,-0.009808,-0.7173,2.3532
median_days_on_market,int64,5559,130,0,51.0000,NaN,58.509624,13.0,157.0


In [16]:
df2 = df[df['month_date_yyyymm'] >= '2017-07-01']

In [17]:
customDescription(df2)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,datetime64[ns],4947,97,0,NaN,NaN,NaN,2017-07-01 00:00:00,2025-07-01 00:00:00
state,object,4947,51,0,NaN,NaN,NaN,NaN,NaN
state_id,object,4947,51,0,NaN,NaN,NaN,NaN,NaN
median_listing_price,int64,4947,2405,0,299900.0000,NaN,372332.786739,144900.0,886500.0
median_listing_price_mm,float64,4947,4333,0,0.0000,NaN,0.005178,-0.103768,0.113545
median_listing_price_yy,float64,4947,2159,0,0.0000,NaN,0.068167,-0.2071,0.4101
active_listing_count,int64,4947,4536,0,NaN,NaN,16732.472205,720.0,182589.0
active_listing_count_mm,float64,4947,4942,0,NaN,NaN,0.000573,-0.307421,0.611643
active_listing_count_yy,float64,4947,3858,0,NaN,NaN,-0.009808,-0.7173,2.3532
median_days_on_market,int64,4947,125,0,44.0000,NaN,56.831817,13.0,152.0


In [18]:
df2[['price_increased_count_yy','pending_listing_count','pending_listing_count_yy','pending_ratio','pending_ratio_yy']] = \
    df2[['price_increased_count_yy','pending_listing_count','pending_listing_count_yy','pending_ratio','pending_ratio_yy']].fillna(0)

In [19]:
customDescription(df2)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,datetime64[ns],4947,97,0,NaN,NaN,NaN,2017-07-01 00:00:00,2025-07-01 00:00:00
state,object,4947,51,0,NaN,NaN,NaN,NaN,NaN
state_id,object,4947,51,0,NaN,NaN,NaN,NaN,NaN
median_listing_price,int64,4947,2405,0,299900.0000,NaN,372332.786739,144900.0,886500.0
median_listing_price_mm,float64,4947,4333,0,0.0000,NaN,0.005178,-0.103768,0.113545
median_listing_price_yy,float64,4947,2159,0,0.0000,NaN,0.068167,-0.2071,0.4101
active_listing_count,int64,4947,4536,0,NaN,NaN,16732.472205,720.0,182589.0
active_listing_count_mm,float64,4947,4942,0,NaN,NaN,0.000573,-0.307421,0.611643
active_listing_count_yy,float64,4947,3858,0,NaN,NaN,-0.009808,-0.7173,2.3532
median_days_on_market,int64,4947,125,0,44.0000,NaN,56.831817,13.0,152.0


In [20]:
# Mapping dictionary: state -> (Region, Division)
state_to_region_division = {
    # Northeast - New England
    "Connecticut": ("Northeast", "New England"),
    "Maine": ("Northeast", "New England"),
    "Massachusetts": ("Northeast", "New England"),
    "New Hampshire": ("Northeast", "New England"),
    "Rhode Island": ("Northeast", "New England"),
    "Vermont": ("Northeast", "New England"),

    # Northeast - Middle Atlantic
    "New Jersey": ("Northeast", "Middle Atlantic"),
    "New York": ("Northeast", "Middle Atlantic"),
    "Pennsylvania": ("Northeast", "Middle Atlantic"),

    # Midwest - East North Central
    "Illinois": ("Midwest", "East North Central"),
    "Indiana": ("Midwest", "East North Central"),
    "Michigan": ("Midwest", "East North Central"),
    "Ohio": ("Midwest", "East North Central"),
    "Wisconsin": ("Midwest", "East North Central"),

    # Midwest - West North Central
    "Iowa": ("Midwest", "West North Central"),
    "Kansas": ("Midwest", "West North Central"),
    "Minnesota": ("Midwest", "West North Central"),
    "Missouri": ("Midwest", "West North Central"),
    "Nebraska": ("Midwest", "West North Central"),
    "North Dakota": ("Midwest", "West North Central"),
    "South Dakota": ("Midwest", "West North Central"),

    # South - South Atlantic
    "Delaware": ("South", "South Atlantic"),
    "Florida": ("South", "South Atlantic"),
    "Georgia": ("South", "South Atlantic"),
    "Maryland": ("South", "South Atlantic"),
    "North Carolina": ("South", "South Atlantic"),
    "South Carolina": ("South", "South Atlantic"),
    "Virginia": ("South", "South Atlantic"),
    "West Virginia": ("South", "South Atlantic"),
    "District of Columbia": ("South", "South Atlantic"),

    # South - East South Central
    "Alabama": ("South", "East South Central"),
    "Kentucky": ("South", "East South Central"),
    "Mississippi": ("South", "East South Central"),
    "Tennessee": ("South", "East South Central"),

    # South - West South Central
    "Arkansas": ("South", "West South Central"),
    "Louisiana": ("South", "West South Central"),
    "Oklahoma": ("South", "West South Central"),
    "Texas": ("South", "West South Central"),

    # West - Mountain
    "Arizona": ("West", "Mountain"),
    "Colorado": ("West", "Mountain"),
    "Idaho": ("West", "Mountain"),
    "Montana": ("West", "Mountain"),
    "Nevada": ("West", "Mountain"),
    "New Mexico": ("West", "Mountain"),
    "Utah": ("West", "Mountain"),
    "Wyoming": ("West", "Mountain"),

    # West - Pacific
    "Alaska": ("West", "Pacific"),
    "California": ("West", "Pacific"),
    "Hawaii": ("West", "Pacific"),
    "Oregon": ("West", "Pacific"),
    "Washington": ("West", "Pacific"),
}

# Create Region and Division columns
df2["Region"] = df2["state"].map(lambda x: state_to_region_division.get(x, (None, None))[0])
df2["Division"] = df2["state"].map(lambda x: state_to_region_division.get(x, (None, None))[1])

In [21]:
df2['month_date_yyyymm'] = pd.to_datetime(df2['month_date_yyyymm'])
df2['year'] = df2['month_date_yyyymm'].dt.year
df2['month'] = df2['month_date_yyyymm'].dt.month

df2.drop(columns=['month_date_yyyymm'], inplace=True, errors='ignore')

In [22]:
df2.to_csv("data/State_Processed.csv", index=False)

In [23]:
customDescription(county_history_df)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,int64,335193,108,0,NaN,NaN,202057.095575,201607.0000,2.025060e+05
county_fips,int64,335193,3141,0,NaN,48261.0,30308.736966,1001.0000,5.604500e+04
county_name,object,335193,3135,0,NaN,"kenedy, tx",NaN,NaN,NaN
median_listing_price,float64,335124,51553,69,149900.0,NaN,265166.374605,1311.0000,1.300000e+07
median_listing_price_mm,float64,297680,11862,37513,0.0,NaN,0.018544,-0.9971,5.703196e+02
median_listing_price_yy,float64,296379,20354,38814,0.0,NaN,0.126707,-0.9968,2.228749e+02
active_listing_count,float64,335148,5922,45,1.0,NaN,290.100818,0.0000,2.325800e+04
active_listing_count_mm,float64,297609,7980,37584,0.0,NaN,0.010556,-1.0000,3.250000e+01
active_listing_count_yy,float64,296233,19328,38960,0.0,NaN,0.067576,-1.0000,9.150000e+01
median_days_on_market,float64,334811,364,382,58.0,NaN,74.105797,1.0000,3.650000e+02


In [24]:
county_history_df['month_date_yyyymm'] = pd.to_datetime(county_history_df['month_date_yyyymm'].astype(str), format='%Y%m')

In [25]:
# Sort by state and month
county_history_df = county_history_df.sort_values(["county_name", "month_date_yyyymm"])

def recompute_mm(df, base_col, mm_col):
    # Compute pct_change
    df[mm_col] = df.groupby("county_name")[base_col].pct_change()

    # Replace inf values with the base_col (new month value)
    inf_mask = df[mm_col].replace([np.inf, -np.inf], np.nan).isna() & df[base_col].notna()
    df.loc[inf_mask, mm_col] = df.loc[inf_mask, base_col]

    return df

In [26]:
for base, mm in base_to_mm.items():
    if base in county_history_df.columns and mm in county_history_df.columns:
        county_history_df = recompute_mm(county_history_df, base, mm)

In [27]:
county_history_df = county_history_df[county_history_df['month_date_yyyymm'] >= '2017-07-01']

In [28]:
customDescription(county_history_df)

,type,count,nunique,null,mode,least_frequent,mean,min,max
month_date_yyyymm,datetime64[ns],298253,96,0,NaN,NaN,NaN,2017-07-01 00:00:00,2025-06-01 00:00:00
county_fips,int64,298253,3141,0,NaN,48261.0,30310.919293,1001.0,56045.0
county_name,object,298253,3135,0,NaN,"kenedy, tx",NaN,NaN,NaN
median_listing_price,float64,298185,49233,68,149900.0,NaN,273759.214286,1311.0,13000000.0
median_listing_price_mm,float64,298251,218369,2,0.0,NaN,15.520888,-0.997088,920000.0
median_listing_price_yy,float64,296379,20354,1874,0.0,NaN,0.126707,-0.9968,222.8749
active_listing_count,float64,298208,5555,45,1.0,NaN,273.890633,0.0,23258.0
active_listing_count_mm,float64,298242,46342,11,0.0,NaN,0.062129,-1.0,3138.0
active_listing_count_yy,float64,296233,19328,2020,0.0,NaN,0.067576,-1.0,91.5
median_days_on_market,float64,297931,364,322,51.0,NaN,72.219007,1.0,365.0
